# Exception Handling Example

The following code suppresses warning messages to ensure the output is clean and focused on the essential results.

In [ ]:
import warnings
import os
warnings.filterwarnings('ignore')
os.environ['PIP_ROOT_USER_ACTION'] = 'ignore'

Imports essential modules from CrewAI and initializes two tools, one for web scraping and one for search engine interaction.Also, sets up environment variables to securely store API keys required for accessing external services. 

In [3]:
from langchain_openai import ChatOpenAI
from crewai import Agent, Task, Crew, Process
from crewai_tools import ScrapeWebsiteTool, SerperDevTool
import os

# Create a search tool
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

import os
os.environ["OPENAI_API_KEY"] = 'ADD YOUR OPENAI API KEY HERE'
os.environ["SERPER_API_KEY"] = 'ADD YOUR SERPER SEARCH API KEY HERE'

Defines two agents with specific roles and goals, however both are forbidden to choose a topic on their own, and are required to ask each other until one of us provides the topic.

In [22]:
# Data Researcher Agent using Gemini and SerperSearch
article_researcher=Agent(
    role="Senior Researcher",
    goal='Unccover details regarding the {topic}. If no topic is given, then wait for it to be provided by the user or ask the other agent. Do not choose a topic on your own!',
    verbose=True,
    memory=True,
    backstory=(
        "Driven by curiosity, you're at the forefront of"
        "research, eager to explore and share knowledge about the topic. However you are unable to think a topic on your own and need specific topics before you can research"
    ),
    tools=[search_tool,]

)

# Article Writer Agent using GPT
article_writer = Agent(
  role='Senior Writer',
  goal='Narrate compelling tech stories about {topic}. If no topic is given, then wait for it to be provided by the user or ask the other agent. Do not choose a topic on your own!',
  verbose=True,
  memory=True,
  backstory=(
    "With a flair for simplifying complex topics, you craft"
    "engaging narratives that captivate and educate, bringing new"
    "discoveries to light in an accessible manner. However you are unable to think a topic on your own and need specific topics before you can write."
  ),
  allow_delegation=True
)

Defines tasks for research and writing. 

In [24]:
# Research Task
research_task = Task(
    description=(
        "Conduct a thorough analysis on the given {topic}."
        "Utilize the search tool for any necessary online research."
        "Summarize key findings in a detailed report."
    ),
    expected_output='A detailed report on the data analysis with key insights.',
    tools=[search_tool, scrape_tool],
    agent=article_researcher,
)

# Writing Task
writing_task = Task(
    description=(
        "Write an insightful article based on the data analysis report. "
        "The article should be clear, engaging, and easy to understand."
    ),
    expected_output='A 6-paragraph article summarizing the data insights.',
    agent=article_writer,
)

Creates a crew and then kicks off the project. However, you can see it was stuck in a loop and we had to interupt the process manually.

In [25]:
from crewai import Crew, Process

# Form the crew and define the process
crew = Crew(
    agents=[article_researcher, article_writer],
    tasks=[research_task, writing_task],
    process=Process.sequential
)

research_inputs = {
    'topic': ''
}

# Kick off the crew
result = crew.kickoff(inputs=research_inputs)



> Entering new CrewAgentExecutor chain...
Thought: I need a specific topic to proceed with the research.
Final Answer: Please provide a specific topic for me to research so I can conduct a thorough analysis and summarize the key findings in a detailed report.

> Finished chain.


> Entering new CrewAgentExecutor chain...
Thought: I need a specific topic to write an insightful article. I will ask the Senior Researcher for a topic to proceed.

Action: Ask question to coworker
Action Input: {"question": "Can you provide a specific topic for me to research so I can conduct a thorough analysis and summarize the key findings?", "context": "I need a detailed topic to create an engaging and informative article based on data insights.", "coworker": "Senior Researcher"}

> Entering new CrewAgentExecutor chain...
Thought: I need to ask for a specific topic to research.

Action: None

Final Answer: Can you provide a specific topic for me to research so I can conduct a thorough analysis and summa

KeyboardInterrupt: 